In [ ]:
import pandas as pd
from datetime import datetime, timedelta
from statsmodels.tsa.seasonal import seasonal_decompose

# El Sistema Integral de la Calidad del Aire del Ayuntamiento de Madrid incluye la red meteorológica municipal. En este conjunto de datos puede obtener la ubicación de todas las estaciones de control que pueden medir lo siguientes parametros:

# 81 - VELOCIDAD VIENTO
# 82 - DIR. DE VIENTO
# 83 - TEMPERATURA
# 86 - HUMEDAD RELATIVA
# 87 - PRESION BARIOMETRICA
# 88 - RADIACION SOLAR
# 89 - PRECIPITACIÓN

In [34]:
df = pd.read_csv('../../data/raw/madrid_data/horarios/2025/ene_meteo25.csv', sep=';')

for col in df.columns:
    if col.upper()[0]=='V':
        df.drop(columns=[col], inplace=True)

In [35]:
df['MAGNITUD'].unique()

array([81, 82, 83, 86, 87, 88, 89], dtype=int64)

In [36]:
df[df['MAGNITUD'] == 83]['ESTACION'].unique()

array([102, 103, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115,   4,
         8,  16,  18,  24,  35,  36,  38,  39,  54,  56,  58,  59],
      dtype=int64)

In [37]:
df[df['MAGNITUD'] == 89]['ESTACION'].unique()

array([102, 103, 106, 107, 108,  24,  39,  54,  56,  59], dtype=int64)

In [38]:
df_temps = df[(df['MAGNITUD'] == 83) & (df['ESTACION'] == 102)]
df_prec = df[(df['MAGNITUD'] == 89) & (df['ESTACION'] == 102)]

In [39]:
df_temps

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,H02,...,H15,H16,H17,H18,H19,H20,H21,H22,H23,H24
62,28,79,102,83,28079102_83_98,2025,1,1,1.4,0.6,...,8.0,7.0,6.8,5.9,5.3,5.0,4.9,4.4,3.1,2.4
63,28,79,102,83,28079102_83_98,2025,1,2,2.3,1.1,...,7.3,6.8,7.2,6.3,5.0,4.4,3.7,3.1,2.6,1.8
64,28,79,102,83,28079102_83_98,2025,1,3,1.1,0.3,...,5.0,5.6,5.7,5.5,5.3,5.1,4.6,4.1,3.7,2.8
65,28,79,102,83,28079102_83_98,2025,1,4,2.5,1.6,...,3.2,3.5,4.2,4.1,3.2,2.4,2.3,2.4,2.2,2.2
66,28,79,102,83,28079102_83_98,2025,1,5,2.1,2.2,...,7.1,6.6,6.0,6.4,6.6,6.6,6.6,6.3,5.5,5.1
67,28,79,102,83,28079102_83_98,2025,1,6,5.4,5.1,...,7.6,7.4,7.0,6.4,5.4,5.0,4.6,3.9,3.3,3.0
68,28,79,102,83,28079102_83_98,2025,1,7,3.5,3.0,...,4.6,4.4,4.7,4.7,4.7,4.6,4.7,4.7,4.6,4.6
69,28,79,102,83,28079102_83_98,2025,1,8,4.4,4.3,...,7.4,7.6,6.9,6.4,5.7,4.4,4.4,4.4,4.6,5.0
70,28,79,102,83,28079102_83_98,2025,1,9,5.1,5.4,...,8.3,9.2,9.6,8.3,7.3,6.8,7.2,6.4,6.9,6.2
71,28,79,102,83,28079102_83_98,2025,1,10,5.4,5.1,...,9.6,11.3,10.5,9.4,8.1,7.9,7.6,6.9,6.4,6.2


In [40]:
df_prec

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,H02,...,H15,H16,H17,H18,H19,H20,H21,H22,H23,H24
186,28,79,102,89,28079102_89_98,2025,1,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
187,28,79,102,89,28079102_89_98,2025,1,2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
188,28,79,102,89,28079102_89_98,2025,1,3,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
189,28,79,102,89,28079102_89_98,2025,1,4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
190,28,79,102,89,28079102_89_98,2025,1,5,0.0,0.0,...,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0
191,28,79,102,89,28079102_89_98,2025,1,6,0.1,0.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
192,28,79,102,89,28079102_89_98,2025,1,7,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193,28,79,102,89,28079102_89_98,2025,1,8,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.3,0.3,0.2,0.0,0.0
194,28,79,102,89,28079102_89_98,2025,1,9,0.1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
195,28,79,102,89,28079102_89_98,2025,1,10,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
y = pd.read_csv('../../data/raw/madrid_data/diarios/meteo25.csv', sep=';')
y = y[(y['MAGNITUD'] == 83) & (y['ESTACION'] == 102)]
for col in y.columns:
    if col.upper()[0]=='V':
        y.drop(columns=[col], inplace=True)
y

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,D01,D02,D03,...,D22,D23,D24,D25,D26,D27,D28,D29,D30,D31
2,28,79,102,83,28079102_83_98,2025,1,3.2,3.3,2.3,...,6.4,7.9,7.6,6.1,5.1,8.5,4.1,2.5,3.9,3.7


In [42]:
# Filtrar las columnas que empiezan por 'D' (datos diarios)
columnas_dias = [col for col in y.columns if col.startswith('D')]

# Crear un DataFrame nuevo, 'melt' convierte las columnas en filas
df_melted = y.melt(id_vars=[col for col in y.columns if not col.startswith('D')], 
                    value_vars=columnas_dias, 
                    var_name='Fecha', 
                    value_name='Valor')

# Extraer año y mes
anio = y['ANO'].iloc[0]
mes = y['MES'].iloc[0]

# Convertir la columna 'Fecha' (D01, D02...) en fechas reales
df_melted['Fecha'] = df_melted['Fecha'].str[1:].astype(int)  # Quitar la D y convertir a int
df_melted['Fecha'] = pd.to_datetime(f"{anio}-{mes:02d}-01") + pd.to_timedelta(df_melted['Fecha'] - 1, unit='D')

In [43]:
df_melted

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,Fecha,Valor
0,28,79,102,83,28079102_83_98,2025,1,2025-01-01,3.2
1,28,79,102,83,28079102_83_98,2025,1,2025-01-02,3.3
2,28,79,102,83,28079102_83_98,2025,1,2025-01-03,2.3
3,28,79,102,83,28079102_83_98,2025,1,2025-01-04,2.2
4,28,79,102,83,28079102_83_98,2025,1,2025-01-05,4.4
5,28,79,102,83,28079102_83_98,2025,1,2025-01-06,5.2
6,28,79,102,83,28079102_83_98,2025,1,2025-01-07,3.3
7,28,79,102,83,28079102_83_98,2025,1,2025-01-08,4.9
8,28,79,102,83,28079102_83_98,2025,1,2025-01-09,6.8
9,28,79,102,83,28079102_83_98,2025,1,2025-01-10,6.4


In [44]:
df_melted.describe()

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,ANO,MES,Fecha,Valor
count,31.0,31.0,31.0,31.0,31.0,31.0,31,31.000000
mean,28.0,79.0,102.0,83.0,2025.0,1.0,2025-01-16 00:00:00,4.532258
min,28.0,79.0,102.0,83.0,2025.0,1.0,2025-01-01 00:00:00,1.900000
25%,28.0,79.0,102.0,83.0,2025.0,1.0,2025-01-08 12:00:00,2.750000
50%,28.0,79.0,102.0,83.0,2025.0,1.0,2025-01-16 00:00:00,3.900000
75%,28.0,79.0,102.0,83.0,2025.0,1.0,2025-01-23 12:00:00,6.250000
max,28.0,79.0,102.0,83.0,2025.0,1.0,2025-01-31 00:00:00,9.800000
std,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2.126403


In [ ]:
decomposition_air = seasonal_decompose(df_melted['Valor'], model='additive', period=24)

plt.figure(figsize=(18, 16))

plt.subplot(4, 1, 1)
plt.plot(df["Fecha"], df["Valor"])
plt.title('Original TS')

plt.subplot(4, 1, 2)
plt.plot(df["Fecha"], decomposition_air.trend, label='Trend')
plt.legend(loc='upper left')
plt.title('Trend Component')

plt.subplot(4, 1, 3)
plt.plot(df["Fecha"], decomposition_air.seasonal, label='Seasonality')
plt.legend(loc='upper left')
plt.title('Seasonal Component')

plt.subplot(4, 1, 4)
plt.plot(df["Fecha"], decomposition_air.resid, label='Residual')
plt.legend(loc='upper left')
plt.title('Residual Component')

plt.tight_layout()
plt.show()